In [ ]:
!pip install llvmlite --ignore-installed

In [ ]:
!pip install mljar-supervised

In [ ]:
from pandas import read_csv, DataFrame, Series
from pathlib import Path

#list(Path('/kaggle/input').rglob('*.*'))

train = read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')
train.info()
test = read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')
test.info()
sample_subm = read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv')
sample_subm.info()

In [ ]:
# Reduce memory

from numpy import float64, float32, int64, int32, dtype

map_dtypes = {
    'int': dtype(int64),
    'float': dtype(float32)
    }

for col in train:
    if train[col].dtype == dtype(int64):
        train[col] = train[col].astype(int32)
    if train[col].dtype == dtype(float64):
        train[col] = train[col].astype(float32)
        
train.info()

In [ ]:
# Simple prepr

train = train.drop(['row_id'], axis=1)

y = train['target']
map_helper = dict(list(zip(
    y.unique(),
    range(len(y.unique()))
)))

y = y.map(map_helper)

X = train.drop(['target'], axis=1)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

automl = AutoML(mode='Explain')
automl.fit(X_train, y_train)

In [ ]:
from pandas import DataFrame, Series, concat

map_helper_rev = dict(list(zip(
    range(len(y.unique())),
    train['target'].unique(),
)))

row_id = test['row_id']
predictions = automl.predict(test.drop('row_id', axis=1))
predictions = DataFrame(predictions)
predictions = predictions[0].map(map_helper_rev)
predictions = concat([row_id, predictions], axis=1)
predictions.columns = ['row_id', 'target']

predictions

